In [75]:
import numpy as np

# Tarea 1 **[25 pts]**

Sea $m\geq n$. Una matriz $T\in\mathbb{R}^{m\times n}$ se llama **triangular north-east** si
$T_{j,k}=0$ para $j>k$.

El objetivo de esta tarea es fabricar una función `qt` que a partir de una matriz $A\in\mathbb{R}^{m\times n}$
calcule una descomposición

\begin{align*}
QA = T
\end{align*}

con $Q\in\mathbb{R}^{m\times m}$ ortonormal y $T\in\mathbb{R}^{m\times n}$ triangular north-east. La función
`qt` debe usar reflexiones de Householder, es decir

\begin{align*}
Q = H_{w_n} \cdot \dots \cdot H_{w_1}.
\end{align*}

En vez de calcular $Q$ explicitamente, la función `qt` calcula la matriz

\begin{align*}
W =
\begin{pmatrix}
| & & |\\
w_1 & \dots & w_n\\
| & & |
\end{pmatrix}.
\end{align*}

La función debe tener la signatura

`(W,T) = qt(A)`.

Las matrices $A,W,T$ son todas del tipo `np.array`.

> Para que la notación sea congruente con lo estudiado en el curso y el apunte, se debe entender $Q$ como
> $$Q = Q_n\cdot\dots\cdot Q_1$$
> donde $Q_k$ se define como
>\begin{align*}
>Q_k =
>\begin{pmatrix}
>I_{k-1} & 0\\
>0 & H
>\end{pmatrix}.
>\end{align*}
> con $H$ la matriz de Householder asociada al paso k de dimensiones adecuadas.
>
> Además se entiende a $W$ como la matriz cuadrada triangular inferior izquierda que contiene los vectores $w_k$ en el orden presentado arriba. Notese que los vectores son de dimensión $n-k+1$, luego la matriz $W$ carece de ciertos elementos, estos se rellenan por arriba por de cada $w_k" con 0 quedando la definición anterior.
>
> Sobre la iteración $k$-ésima, si el vector en cuestión es múltiplo de $e_1$, entonces la matriz de Householder debe ser $I$, por lo tanto en tal caso asumiremos $w_k=0$ el vector nulo.


In [76]:
def printif(text ,var=1):
    '''print by a condition'''
    if var == 1:
        print(text)

def qt(A, verbose=0):
    '''Return a representation of the factorization QA=T with Q mxm orthonormal and T matrix mxn triangular north-east
     where W=(w_1,...,w_n)'''
    n, m = np.shape(A) # dimension
    # m<n error
    if m < n:
        raise ValueError("m must be greater or equal than n")
    A = A.astype(np.float64)
    W = np.zeros([n,n], dtype=np.float64)
    # Over iteration
    for k in range(n-1):
        printif(f"-------------  k = {k}  -------------", verbose)
        x_k = np.copy(A[k:,k])
        # if it is trivial case: x_1 = b*e_1, we asume w_k = 0
        if np.round(np.linalg.norm(x_k[1:], ord=2), 10) != 0:
            # lema 27
            lmbda = -np.sign(x_k[0])
            sgm = lmbda*np.linalg.norm(x_k, ord=2)
            e_1 = np.eye(len(x_k), 1, dtype=int).flatten()
            W[k:,k] = x_k+sgm*e_1
            W[k:,k] /= np.linalg.norm(W[k:,k], ord=2)            
        printif(f"A:\n{A}", verbose)
        # Calculate the new value of A by HB
        printif(f"B:\n{A[k:,k:]}", verbose)
        wHB= np.matmul(np.transpose(np.reshape(W[k:,k],(-1,1))), A[k:,k:])
        A[k:,k:] = A[k:,k:] - 2*np.matmul(np.reshape(W[k:,k],(-1,1)), wHB) #w have norm 1 or 0
        printif(f"W:\n{W}", verbose)
    return (np.round(W, 10), np.round(A, 10))


# A = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],])
    
A = np.array([[1,-1,-1],[2,0,1],[-2,7,1]]) 

W, T = qt(A)

print(f"Las matrices resultantes son:\nW:\n{W}\nT:\n{T}\nA:\n{A}")

Las matrices resultantes son:
W:
[[-0.57735027  0.          0.        ]
 [ 0.57735027 -0.31622777  0.        ]
 [-0.57735027  0.9486833   0.        ]]
T:
[[ 3.         -5.         -0.33333333]
 [-0.          5.          1.26666667]
 [ 0.         -0.         -1.13333333]]
A:
[[ 1 -1 -1]
 [ 2  0  1]
 [-2  7  1]]


# Tarea 2 **[25 pts]**

En un segundo paso, escriba una función

`(y) = evalHouseholder(W,x)`

que calcule $y = Qx$, donde $W$ representa a la matriz ortogonal $Q$ según las indicaciones de arriba.
La única restricción es que la función `evalHouseholder` debe pedir solamente la memoria para el vector $y$.

In [77]:
def evalHouseholder(W,x):
    
    m = W.shape[0];
    y = np.zeros(m);
    
    #
    # implemente aqui la función SIN pedir mas memoria para matrices o vectores
    #
    
    return y

# Tarea 3 **[25 pts]**


En un tercer paso aplicamos las funciones de arriba al problema de calcular $x\in\mathbb{R}^n$ tal que

\begin{align*}
\| b-Ax \|_2 = \min_{y\in\mathbb{R}^n}\| b-Ay \|_2
\end{align*}

**sin** usar las ecuaciones normales. Aplicaremos la técnica del Lemma 30 de los apuntes.

In [78]:
def minquad(A,b):
    
    (W,T) = qt(A);
    
    #
    #  implemente aqui la función
    #
    
    return x;

# Tarea 4 **[25 pts]**

Invente unos ejemplos para testear sus funciones!